In [0]:
%sql
use catalog uc_prod;
use schema silver

In [0]:
import pandas as pd
pd_series = pd.date_range(start='2022-01-01', end='2025-12-31', freq='1D')
pd_df = pd.DataFrame({'date':pd_series})

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark_df  = spark.createDataFrame(pd_df)
final_df = spark_df.withColumn('date',to_date(date_format('date','yyyy-MM-dd')))\
        .withColumn('date_id',date_format('date','yyyyMMdd'))

In [0]:
# Register as a temporary SQL view
final_df.createOrReplaceTempView("date_table")


In [0]:
%sql
insert into gold.reporting_dim_date_gold (
CALENDAR_DATE,DATE_ID,lakehouse_inserted_date,lakehouse_updated_date
)
select st.date as CALENDAR_DATE,st.date_id as DATE_ID,current_timestamp() as lakehouse_inserted_date,current_timestamp() as lakehouse_updated_date 
from date_table as st

In [0]:
%sql
insert into uc_prod.watermark_tbl.process_logs
(Process_name,Process_file_updated_date,process_status)
values
(
'reporting_dim_calender_gold',
current_timestamp(),
'completed'
)
